In [90]:
from mdutils.mdutils import MdUtils
import tqdm
import math
import requests
import pandas as pd
import time
from wikidataintegrator import wdi_core, wdi_login
from rdflib import Graph, Literal, RDF, URIRef, Namespace, BNode
from rdflib.namespace import DCTERMS, FOAF, XSD, DC, RDFS
import shutil
import os

# Load occurence data

In [91]:
df = pd.read_csv('data/0002020-240626123714530/occurrence.txt', sep='\t', on_bad_lines = 'warn')
df

,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,...,publishedByGbifRegion,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name,iucnRedListCategory
0,4876300936,NaN,NaN,NaN,CC_BY_4_0,2023-03-20T00:00:00Z,NaN,NaN,Meise Botanic Garden,PhysicalObject,...,EUROPE,MSR,Montserrat,MSR.3_1,Saint Peter,NaN,NaN,NaN,NaN,LC
1,4876284841,NaN,NaN,NaN,CC_BY_4_0,2023-03-21T00:00:00Z,NaN,NaN,Meise Botanic Garden,PhysicalObject,...,EUROPE,MSR,Montserrat,MSR.3_1,Saint Peter,NaN,NaN,NaN,NaN,NE
2,4607368012,NaN,NaN,NaN,CC_BY_NC_4_0,2024-05-24T09:26:47Z,NaN,https://www.inaturalist.org/observations/20511...,slauren,NaN,...,NORTH_AMERICA,MSR,Montserrat,MSR.3_1,Saint Peter,NaN,NaN,NaN,NaN,NE
3,4536015362,NaN,NaN,NaN,CC0_1_0,2024-04-26T16:57:26Z,NaN,https://www.inaturalist.org/observations/20102...,monkeyjodey,NaN,...,EUROPE,MSR,Montserrat,MSR.3_1,Saint Peter,NaN,NaN,NaN,NaN,NE
4,4536000017,NaN,NaN,NaN,CC0_1_0,2024-04-26T16:57:26Z,NaN,https://www.inaturalist.org/observations/20125...,monkeyjodey,NaN,...,EUROPE,MSR,Montserrat,MSR.1_1,Saint Anthon,NaN,NaN,NaN,NaN,NE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,1319347135,NaN,NaN,NaN,CC0_1_0,2017-10-31T12:14:00Z,NaN,NaN,NaN,NaN,...,NORTH_AMERICA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LC
83,1318020074,NaN,NaN,NaN,CC0_1_0,2017-10-31T12:10:00Z,NaN,NaN,NaN,NaN,...,NORTH_AMERICA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NE
84,1317614158,NaN,NaN,NaN,CC0_1_0,2021-03-06T11:43:00Z,NaN,NaN,NaN,NaN,...,NORTH_AMERICA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NE
85,1317605282,NaN,NaN,NaN,CC0_1_0,2021-03-08T09:38:00Z,NaN,NaN,NaN,NaN,...,NORTH_AMERICA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NE


# Fetch institute QID's for institutes

In [92]:
user_agent = 'TaxonQueries/1.0 (andra@micelio.be) SPARQL queries'  # Replace with your actual User-Agent string

result_df = pd.DataFrame(columns=['Institute', 'Institute_QID'])
batchSize = 5
retry_wait_time = 1  # Initial wait time in seconds
institutes = set(df["rightsHolder"])
print(institutes)
instituteQID = dict()
i = 0
notFound = {}
for institute in institutes:
    en = wdi_core.WDItemEngine.get_wd_search_results(institute, language='en')
    if len(en) > 0:
        instituteQID[institute] = en[0]
        continue
    nl = wdi_core.WDItemEngine.get_wd_search_results(institute, language='nl')
    if len(nl) > 0:
        instituteQID[institute] = nl[0]
        continue
    de = wdi_core.WDItemEngine.get_wd_search_results(institute, language='de')
    if len(de) > 0:
        instituteQID[institute] = de[0]
        continue
    fr = wdi_core.WDItemEngine.get_wd_search_results(institute, language='fr')
    if len(fr) > 0:
        instituteQID[institute] = fr[0]
        continue
    es = wdi_core.WDItemEngine.get_wd_search_results(institute, language='es')
    if len(es) > 0:
        instituteQID[institute] = es[0]
        continue
    pt = wdi_core.WDItemEngine.get_wd_search_results(institute, language='pt')
    if len(pt) > 0:
        instituteQID[institute] = pt[0]
        continue
    else:
        i+=1
        print(str(i)+f" Not found institute={institute}")
        notFound[institute] = requests.get("https://api.gbif.org/v1/organization?q="+institute).json()

{'University of Florida', 'Quentin Groom', 'Carnegie Museum of Natural History', 'The New York Botanical Garden', 'Sofie Meeus', nan, 'Delmaude C Ryan', 'Meise Botanic Garden', 'slauren', 'classical', 'gorillarebecca', 'monkeyjodey'}
1 Not found institute=Delmaude C Ryan
2 Not found institute=slauren
3 Not found institute=gorillarebecca
4 Not found institute=monkeyjodey


In [93]:
df2 = df[df["taxonRank"] == "SPECIES"]
df3 = df2[["species", "taxonKey", "taxonRank"]].drop_duplicates()
species_counts = df3["species"].value_counts()
species_counts

grouped = df3.groupby('species').apply(lambda x: list(x.taxonKey))
for species, taxon_keys in grouped.items():
    if len(taxon_keys) > 1:
        taxon_keys = list(map(int, taxon_keys))
        print(f"Species: {species}, Taxon Keys: {taxon_keys}")

/var/folders/nt/bqljt7p919zgfjwhldlsbdvh0000gn/T/ipykernel_69164/69063171.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = df3.groupby('species').apply(lambda x: list(x.taxonKey))


In [94]:
taxonname =  df2[["species", "taxonKey", "taxonRank", "taxonRank", "genus", "genericName","taxonomicStatus", "acceptedNameUsageID", "scientificName", "specificEpithet"]].drop_duplicates()
taxonnames = []
for index, row in taxonname.iterrows():
    if isinstance(row["specificEpithet"], str):
        taxonnames.append("\""+row["genericName"] + " " + row["specificEpithet"]+"\"")
    else:
        continue
    taxonnames.append("\""+row["genericName"] + " " + row["specificEpithet"]+"\"")
taxonnames = list(set(taxonnames))

# resolve taxonnames and kingdom

In [95]:
url = 'https://query.wikidata.org/sparql'
user_agent = 'TaxonQueries/1.0 ( andra@micelio.be) SPARQL queries'  # Replace with your actual User-Agent string

result_df = pd.DataFrame(columns=['taxonId', 'taxonname', 'taxon', 'kingdomname', 'kingdom' ])
batchSize = 10
retry_wait_time = 1  # Initial wait time in seconds

for i in tqdm.tqdm(range(math.ceil(len(taxonnames)/batchSize)), desc="Processing batches"):
    batch = taxonnames[i*batchSize:(i+1)*batchSize]
    query = """
            SELECT ?taxon ?taxonId ?taxonname ?kingdomname ?kingdom ?article
                WHERE {
                    VALUES ?taxonname {""" + " ".join(batch) + """}
                    ?taxon wdt:P225 ?taxonname .
                    ?taxon wdt:P171* ?kingdom .
                    OPTIONAL {?article schema:about ?taxon ;
                               schema:isPartOf <https://en.wikipedia.org/> .}
                    ?kingdom wdt:P105 wd:Q36732 .
                    ?kingdom rdfs:label ?kingdomname filter (lang(?kingdomname) = "en") .
                    OPTIONAL {?taxon wdt:P846 ?taxonId .}
            }
            """
    # print(query)

    success = False
    while not success:
        try:
            headers = {'User-Agent': user_agent}
            r = requests.get(url, params={'format': 'json', 'query': query}, headers=headers)
            r.raise_for_status()  # Raise an HTTPError for bad responses
            data = r.json()
            temp_df = pd.DataFrame([(item.get("taxonId", {}).get("value", ""),
                                     item.get("taxonname", {}).get("value", ""),
                                     item.get("taxon", {}).get("value", ""),
                                     item.get("kingdomname", {}).get("value", ""),
                                     item.get("kingdom", {}).get("value", ""),
                                     item.get("article", {}).get("value", "")
                                    )
                                    for item in data["results"]["bindings"]],
                                    columns=['taxonId', 'taxonname', 'taxon', 'kingdomname', 'kingdom', 'article' ])

            # Concatenate the temporary DataFrame with the main result DataFrame
            result_df = pd.concat([result_df, temp_df], ignore_index=True)
            success = True  # Set to True to break out of the while loop on success

        except requests.exceptions.HTTPError as err:
            print(f"Error: {err}")
            print(f"Retrying in {retry_wait_time} seconds...")
            time.sleep(retry_wait_time)
            retry_wait_time *= 2  # Double the wait time
            if retry_wait_time > 60:  # Limit the maximum wait time to 60 seconds
                retry_wait_time = 60
        else:
            retry_wait_time = 1  # Reset to 1 second after a successful query

# Now, result_df contains all the data
result_df


Processing batches: 100%|██████████| 3/3 [00:01<00:00,  2.38it/s]


,taxonId,taxonname,taxon,kingdomname,kingdom,article
0,5289739,Cenchrus echinatus,http://www.wikidata.org/entity/Q4925284,plant,http://www.wikidata.org/entity/Q756,https://en.wikipedia.org/wiki/Cenchrus_echinatus
1,2704745,Sporobolus virginicus,http://www.wikidata.org/entity/Q7579252,plant,http://www.wikidata.org/entity/Q756,https://en.wikipedia.org/wiki/Sporobolus_virgi...
2,2717421,Eleocharis flavescens,http://www.wikidata.org/entity/Q15584858,plant,http://www.wikidata.org/entity/Q756,https://en.wikipedia.org/wiki/Eleocharis_flave...
3,2702504,Melinis repens,http://www.wikidata.org/entity/Q24700355,plant,http://www.wikidata.org/entity/Q756,https://en.wikipedia.org/wiki/Melinis_repens
4,2705068,Panicum trichoides,http://www.wikidata.org/entity/Q10912779,plant,http://www.wikidata.org/entity/Q756,
5,2708680,Fimbristylis complanata,http://www.wikidata.org/entity/Q11075670,plant,http://www.wikidata.org/entity/Q756,
6,2702854,Lasiacis sorghoidea,http://www.wikidata.org/entity/Q15508551,plant,http://www.wikidata.org/entity/Q756,
7,2702852,Lasiacis divaricata,http://www.wikidata.org/entity/Q15508581,plant,http://www.wikidata.org/entity/Q756,
8,2702522,Eriochloa punctata,http://www.wikidata.org/entity/Q15516186,plant,http://www.wikidata.org/entity/Q756,
9,5291147,Machaerina restioides,http://www.wikidata.org/entity/Q15550242,plant,http://www.wikidata.org/entity/Q756,


# Merge taxonnames and kingdom

In [96]:
result_df[['genericName', 'specificEpithet']] = result_df['taxonname'].str.split(expand=True, n=1)

# Print the updated DataFrame
wdtaxqid = dict()
for index, row in result_df.iterrows():
    if row["taxonId"] != "":
        wdtaxqid[row["taxonId"]] = row["taxon"]
result_df

,taxonId,taxonname,taxon,kingdomname,kingdom,article,genericName,specificEpithet
0,5289739,Cenchrus echinatus,http://www.wikidata.org/entity/Q4925284,plant,http://www.wikidata.org/entity/Q756,https://en.wikipedia.org/wiki/Cenchrus_echinatus,Cenchrus,echinatus
1,2704745,Sporobolus virginicus,http://www.wikidata.org/entity/Q7579252,plant,http://www.wikidata.org/entity/Q756,https://en.wikipedia.org/wiki/Sporobolus_virgi...,Sporobolus,virginicus
2,2717421,Eleocharis flavescens,http://www.wikidata.org/entity/Q15584858,plant,http://www.wikidata.org/entity/Q756,https://en.wikipedia.org/wiki/Eleocharis_flave...,Eleocharis,flavescens
3,2702504,Melinis repens,http://www.wikidata.org/entity/Q24700355,plant,http://www.wikidata.org/entity/Q756,https://en.wikipedia.org/wiki/Melinis_repens,Melinis,repens
4,2705068,Panicum trichoides,http://www.wikidata.org/entity/Q10912779,plant,http://www.wikidata.org/entity/Q756,,Panicum,trichoides
5,2708680,Fimbristylis complanata,http://www.wikidata.org/entity/Q11075670,plant,http://www.wikidata.org/entity/Q756,,Fimbristylis,complanata
6,2702854,Lasiacis sorghoidea,http://www.wikidata.org/entity/Q15508551,plant,http://www.wikidata.org/entity/Q756,,Lasiacis,sorghoidea
7,2702852,Lasiacis divaricata,http://www.wikidata.org/entity/Q15508581,plant,http://www.wikidata.org/entity/Q756,,Lasiacis,divaricata
8,2702522,Eriochloa punctata,http://www.wikidata.org/entity/Q15516186,plant,http://www.wikidata.org/entity/Q756,,Eriochloa,punctata
9,5291147,Machaerina restioides,http://www.wikidata.org/entity/Q15550242,plant,http://www.wikidata.org/entity/Q756,,Machaerina,restioides


In [97]:
merged_df = pd.merge(df, result_df, on=['genericName', 'specificEpithet'], how='inner')
merged_df

,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,...,level2Name,level3Gid,level3Name,iucnRedListCategory,taxonId,taxonname,taxon,kingdomname,kingdom_y,article
0,4876300936,NaN,NaN,NaN,CC_BY_4_0,2023-03-20T00:00:00Z,NaN,NaN,Meise Botanic Garden,PhysicalObject,...,NaN,NaN,NaN,LC,2718767,Cyperus brevifolius,http://www.wikidata.org/entity/Q21265100,plant,http://www.wikidata.org/entity/Q756,
1,4876284841,NaN,NaN,NaN,CC_BY_4_0,2023-03-21T00:00:00Z,NaN,NaN,Meise Botanic Garden,PhysicalObject,...,NaN,NaN,NaN,NE,2721307,Rhynchospora nervosa,http://www.wikidata.org/entity/Q15555289,plant,http://www.wikidata.org/entity/Q756,
2,4607368012,NaN,NaN,NaN,CC_BY_NC_4_0,2024-05-24T09:26:47Z,NaN,https://www.inaturalist.org/observations/20511...,slauren,NaN,...,NaN,NaN,NaN,NE,2702504,Melinis repens,http://www.wikidata.org/entity/Q24700355,plant,http://www.wikidata.org/entity/Q756,https://en.wikipedia.org/wiki/Melinis_repens
3,4536015362,NaN,NaN,NaN,CC0_1_0,2024-04-26T16:57:26Z,NaN,https://www.inaturalist.org/observations/20102...,monkeyjodey,NaN,...,NaN,NaN,NaN,NE,2702504,Melinis repens,http://www.wikidata.org/entity/Q24700355,plant,http://www.wikidata.org/entity/Q756,https://en.wikipedia.org/wiki/Melinis_repens
4,4536000017,NaN,NaN,NaN,CC0_1_0,2024-04-26T16:57:26Z,NaN,https://www.inaturalist.org/observations/20125...,monkeyjodey,NaN,...,NaN,NaN,NaN,NE,2702504,Melinis repens,http://www.wikidata.org/entity/Q24700355,plant,http://www.wikidata.org/entity/Q756,https://en.wikipedia.org/wiki/Melinis_repens
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,1319347135,NaN,NaN,NaN,CC0_1_0,2017-10-31T12:14:00Z,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,LC,2712691,Scleria secans,http://www.wikidata.org/entity/Q15564658,plant,http://www.wikidata.org/entity/Q756,
83,1318020074,NaN,NaN,NaN,CC0_1_0,2017-10-31T12:10:00Z,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NE,2715182,Cyperus sphacelatus,http://www.wikidata.org/entity/Q15535602,plant,http://www.wikidata.org/entity/Q756,https://en.wikipedia.org/wiki/Cyperus_sphacelatus
84,1317614158,NaN,NaN,NaN,CC0_1_0,2021-03-06T11:43:00Z,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NE,2705827,Brachiaria fasciculata,http://www.wikidata.org/entity/Q15509620,plant,http://www.wikidata.org/entity/Q756,
85,1317605282,NaN,NaN,NaN,CC0_1_0,2021-03-08T09:38:00Z,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NE,2702854,Lasiacis sorghoidea,http://www.wikidata.org/entity/Q15508551,plant,http://www.wikidata.org/entity/Q756,


In [98]:
merged_df[['taxonKey', 'taxonId', 'taxonname', 'taxon', 'genericName', 'specificEpithet', 'kingdom_y', 'kingdomname', 'article']].drop_duplicates()

,taxonKey,taxonId,taxonname,taxon,genericName,specificEpithet,kingdom_y,kingdomname,article
0,2718767,2718767,Cyperus brevifolius,http://www.wikidata.org/entity/Q21265100,Cyperus,brevifolius,http://www.wikidata.org/entity/Q756,plant,
1,2721307,2721307,Rhynchospora nervosa,http://www.wikidata.org/entity/Q15555289,Rhynchospora,nervosa,http://www.wikidata.org/entity/Q756,plant,
2,2702504,2702504,Melinis repens,http://www.wikidata.org/entity/Q24700355,Melinis,repens,http://www.wikidata.org/entity/Q756,plant,https://en.wikipedia.org/wiki/Melinis_repens
5,7661971,2705758,Bambusa vulgaris,http://www.wikidata.org/entity/Q3219428,Bambusa,vulgaris,http://www.wikidata.org/entity/Q756,plant,https://en.wikipedia.org/wiki/Bambusa_vulgaris
9,5289739,5289739,Cenchrus echinatus,http://www.wikidata.org/entity/Q4925284,Cenchrus,echinatus,http://www.wikidata.org/entity/Q756,plant,https://en.wikipedia.org/wiki/Cenchrus_echinatus
11,2702852,2702852,Lasiacis divaricata,http://www.wikidata.org/entity/Q15508581,Lasiacis,divaricata,http://www.wikidata.org/entity/Q756,plant,
15,2715182,2715182,Cyperus sphacelatus,http://www.wikidata.org/entity/Q15535602,Cyperus,sphacelatus,http://www.wikidata.org/entity/Q756,plant,https://en.wikipedia.org/wiki/Cyperus_sphacelatus
16,2704745,2704745,Sporobolus virginicus,http://www.wikidata.org/entity/Q7579252,Sporobolus,virginicus,http://www.wikidata.org/entity/Q756,plant,https://en.wikipedia.org/wiki/Sporobolus_virgi...
19,2705924,2705924,Axonopus compressus,http://www.wikidata.org/entity/Q4830676,Axonopus,compressus,http://www.wikidata.org/entity/Q756,plant,https://en.wikipedia.org/wiki/Axonopus_compressus
22,2717421,2717421,Eleocharis flavescens,http://www.wikidata.org/entity/Q15584858,Eleocharis,flavescens,http://www.wikidata.org/entity/Q756,plant,https://en.wikipedia.org/wiki/Eleocharis_flave...


In [99]:
taxoninformation = dict()

In [100]:
def get_wikipedia_snippet(title):
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "extracts",
        "exintro": True,
        "explaintext": True,
        "redirects": 1
    }
    for attempt in range(3):
        try:
            response = requests.get(url, params=params, timeout=15)
            if response.status_code != 200 or not response.text.strip():
                time.sleep(2 ** attempt)
                continue
            data = response.json()
            page = next(iter(data["query"]["pages"].values()))
            return page.get("extract", "").strip()
        except Exception:
            time.sleep(2 ** attempt)
    return ""



In [101]:
# Assuming `result_df` is your DataFrame and it has columns named 'article' and 'taxonname'
taxonname_list = result_df[result_df["article"] != ""]["taxonname"].tolist()
taxonname_list

['Cenchrus echinatus',
 'Sporobolus virginicus',
 'Eleocharis flavescens',
 'Melinis repens',
 'Cymbopogon citratus',
 'Bambusa vulgaris',
 'Paspalum conjugatum',
 'Aristida adscensionis',
 'Anthephora hermaphrodita',
 'Cyperus sphacelatus',
 'Coix lacryma-jobi',
 'Axonopus compressus',
 'Cyperus alopecuroides',
 'Digitaria insularis']

In [102]:
for taxonname in tqdm.tqdm(taxonname_list):
    if taxonname not in taxoninformation.keys():
        taxoninformation[taxonname] = dict()
        try:
            taxoninformation[taxonname]["wikiintroduction"] = get_wikipedia_snippet(taxonname.replace(" ", "_"))
        except Exception as e:
            print(f"Warning: could not fetch Wikipedia snippet for {taxonname}: {e}")
            taxoninformation[taxonname]["wikiintroduction"] = ""
        time.sleep(0.5)  # be polite to the Wikipedia API


100%|██████████| 14/14 [00:04<00:00,  3.33it/s]


In [103]:
taxon_info_df = pd.DataFrame.from_dict(taxoninformation, orient='index').reset_index()
taxon_info_df.rename(columns={'index': 'taxonname'}, inplace=True)

# Merge the dictionary DataFrame with your existing DataFrame
merged_df2 = merged_df.merge(taxon_info_df, on='taxonname', how='left')
merged_df2

,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,...,level3Gid,level3Name,iucnRedListCategory,taxonId,taxonname,taxon,kingdomname,kingdom_y,article,wikiintroduction
0,4876300936,NaN,NaN,NaN,CC_BY_4_0,2023-03-20T00:00:00Z,NaN,NaN,Meise Botanic Garden,PhysicalObject,...,NaN,NaN,LC,2718767,Cyperus brevifolius,http://www.wikidata.org/entity/Q21265100,plant,http://www.wikidata.org/entity/Q756,,NaN
1,4876284841,NaN,NaN,NaN,CC_BY_4_0,2023-03-21T00:00:00Z,NaN,NaN,Meise Botanic Garden,PhysicalObject,...,NaN,NaN,NE,2721307,Rhynchospora nervosa,http://www.wikidata.org/entity/Q15555289,plant,http://www.wikidata.org/entity/Q756,,NaN
2,4607368012,NaN,NaN,NaN,CC_BY_NC_4_0,2024-05-24T09:26:47Z,NaN,https://www.inaturalist.org/observations/20511...,slauren,NaN,...,NaN,NaN,NE,2702504,Melinis repens,http://www.wikidata.org/entity/Q24700355,plant,http://www.wikidata.org/entity/Q756,https://en.wikipedia.org/wiki/Melinis_repens,Melinis repens is a species of grass known by ...
3,4536015362,NaN,NaN,NaN,CC0_1_0,2024-04-26T16:57:26Z,NaN,https://www.inaturalist.org/observations/20102...,monkeyjodey,NaN,...,NaN,NaN,NE,2702504,Melinis repens,http://www.wikidata.org/entity/Q24700355,plant,http://www.wikidata.org/entity/Q756,https://en.wikipedia.org/wiki/Melinis_repens,Melinis repens is a species of grass known by ...
4,4536000017,NaN,NaN,NaN,CC0_1_0,2024-04-26T16:57:26Z,NaN,https://www.inaturalist.org/observations/20125...,monkeyjodey,NaN,...,NaN,NaN,NE,2702504,Melinis repens,http://www.wikidata.org/entity/Q24700355,plant,http://www.wikidata.org/entity/Q756,https://en.wikipedia.org/wiki/Melinis_repens,Melinis repens is a species of grass known by ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,1319347135,NaN,NaN,NaN,CC0_1_0,2017-10-31T12:14:00Z,NaN,NaN,NaN,NaN,...,NaN,NaN,LC,2712691,Scleria secans,http://www.wikidata.org/entity/Q15564658,plant,http://www.wikidata.org/entity/Q756,,NaN
83,1318020074,NaN,NaN,NaN,CC0_1_0,2017-10-31T12:10:00Z,NaN,NaN,NaN,NaN,...,NaN,NaN,NE,2715182,Cyperus sphacelatus,http://www.wikidata.org/entity/Q15535602,plant,http://www.wikidata.org/entity/Q756,https://en.wikipedia.org/wiki/Cyperus_sphacelatus,Cyperus sphacelatus is a species of sedge that...
84,1317614158,NaN,NaN,NaN,CC0_1_0,2021-03-06T11:43:00Z,NaN,NaN,NaN,NaN,...,NaN,NaN,NE,2705827,Brachiaria fasciculata,http://www.wikidata.org/entity/Q15509620,plant,http://www.wikidata.org/entity/Q756,,NaN
85,1317605282,NaN,NaN,NaN,CC0_1_0,2021-03-08T09:38:00Z,NaN,NaN,NaN,NaN,...,NaN,NaN,NE,2702854,Lasiacis sorghoidea,http://www.wikidata.org/entity/Q15508551,plant,http://www.wikidata.org/entity/Q756,,NaN


In [104]:
gbifRDF = Graph()
WD = Namespace("http://www.wikidata.org/entity/")
WDT = Namespace("http://www.wikidata.org/prop/direct/")
gbifRDF.bind("wd", WD)
gbifRDF.bind("wdt", WDT)
gbifRDF.bind("dcterms", DCTERMS)
gbifRDF.bind("geo", Namespace("http://www.opengis.net/ont/geosparql#"))

for index, row in tqdm.tqdm(merged_df2.iterrows()):
    GBIFURI = URIRef("https://gbif.semscape.org/occurrence/"+str(row["gbifID"]))
    gbifRDF.add((GBIFURI, WDT.P854 , URIRef("https://www.gbif.org/occurrence/"+str(row["gbifID"]))))
    gbifRDF.add((GBIFURI, DCTERMS.identifier , Literal(str(row["gbifID"]))))

    # species
    tempkey = str(row['taxonKey']).replace('.0', '')
    if tempkey in wdtaxqid.keys():
        gbifRDF.add((GBIFURI, WDT.P225, URIRef(wdtaxqid[tempkey])))
        gbifRDF.add((URIRef(wdtaxqid[tempkey]), RDFS.label, Literal(row["taxonname"], lang="en")))

    # lat long
    try:
        if not math.isnan(float(row["decimalLatitude"])):
            gbifRDF.add((GBIFURI, WDT.P625, Literal("Point("+str(row["decimalLongitude"])+" "+str(row["decimalLatitude"])+")", datatype="http://www.opengis.net/ont/geosparql#wktLiteral")))
    except (TypeError, ValueError) as e:
        print(f"Error for gbifID={row['gbifID']}, Latitude={row['decimalLatitude']}, Longitude={row['decimalLongitude']}")
        print(f"Error details: {e}")
        print(f"Error occurred in this line: {e.__traceback__.tb_lineno}")

    # type
    typeDict = {
    "Collection": "Q2668072","Colletion": "Q2668072","collection": "Q2668072",
    "Event": "Q1656682",
    "Objeto físico": "Q223557","Physical Object": "Q223557","Physical object": "Q223557","PhysicalObject": "Q223557","PhysicalSpecimen": "Q223557","http://purl.org/dc/dcmitype/PhysicalObject": "Q223557",
    "Sound": "Q11442",
    "StillImage": "Q478798","http://purl.org/dc/dcmitype/StillImage": "Q478798",
    "Text": "Q1145976","text": "Q1145976"
}
    if row["type"] in typeDict.keys():
        gbifRDF.add((GBIFURI, WDT.P31, URIRef("http://www.wikidata.org/entity/"+typeDict[row["type"]])))
    else:
        pass
        # print(f"Error for gbifID={row['gbifID']}, type={row['type']}")

    # language
    langDict = {
        "EN": "Q1860", "en": "Q1860","En": "Q1860", "English": "Q1860",
        "PT": "Q5146","português": "Q5146",
        "es": "Q1321",
        "nl": "Q7411",
        "fr": "Q150", "FR": "Q150",
        "Latin": "Q397", "la": "Q397"
    }

    if not pd.isna(row["language"]) and row["language"] in langDict:
        gbifRDF.add((GBIFURI, WDT.P407, URIRef("http://www.wikidata.org/entity/"+langDict[row["language"]])))
    else:
        pass
        # print(f"Error for gbifID={row['gbifID']}, language={row['language']}")

    # IUCN Red List Category
    iucnDict = {
        "NE": "Q3350324", "Not Evaluated": "Q3350324",
        "LC": "Q211005", "Least Concern": "Q211005",
        "NT": "Q719675", "Near Threatened": "Q719675",
        "VU": "Q278113", "Vulnerable": "Q278113",
        "EN": "Q11394", "Endangered": "Q11394",
        "CR": "Q219127", "Critically Endangered": "Q219127",
        "EW": "Q239509", "Extinct in the Wild": "Q239509",
        "EX": "Q237350", "Extinct": "Q237350",
        "DD": "Q3245245", "Data Deficient": "Q3245245"
    }
    if not pd.notna(row["iucnRedListCategory"]) and row["iucnRedListCategory"] in iucnDict:
        gbifRDF.add((GBIFURI, WDT.P141, URIRef("http://www.wikidata.org/entity/"+iucnDict[row["iucnRedListCategory"]])))
    else:
        pass
        # print(f"Error for gbifID={row['gbifID']}, iucnRedListCategory={row['iucnRedListCategory']}")

    # taxonank
    taxonrankDict = {
        "class": "Q37517", "Class": "Q37517",
        "family": "Q35409", "Family": "Q35409",
        "genus": "Q34740", "Genus": "Q34740",
        "kingdom": "Q36732", "Kingdom": "Q36732",
        "order": "Q36602", "Order": "Q36602",
        "phylum": "Q38348", "Phylum": "Q38348",
        "species": "Q7432", "Species": "Q7432",
        "subspecies": "Q68947", "Subspecies": "Q68947",
        "variety": "Q767728", "Variety": "Q767728",
        "form": "Q279749", "Form": "Q279749"
    }

    if not pd.isna(row["taxonRank"]) and row["taxonRank"] in taxonrankDict:
        gbifRDF.add((GBIFURI, WDT.P105, URIRef("http://www.wikidata.org/entity/"+taxonrankDict[row["taxonRank"]])))
    else:
        pass
        # print(f"Error for gbifID={row['gbifID']}, taxonRank={row['taxonRank']}")
    # license
    licenseDict = {
        "CC_BY_NC_ND_2_5": "Q19113746", "http://creativecommons.org/licenses/by-nc-nd/2.5/": "Q19113746",
        "CC_BY_NC_ND_4_0": "Q24082749", "http://creativecommons.org/licenses/by-nc-nd/4.0/": "Q24082749",
        "CC_BY_NC_SA_2_5": "Q19068212", "http://creativecommons.org/licenses/by-nc-sa/2.5/": "Q19068212",
        "CC_BY_NC_SA_3_0": "Q15643954", "http://creativecommons.org/licenses/by-nc-sa/3.0/": "Q15643954",
        "CC_BY_NC_SA_4_0": "Q42553662", "http://creativecommons.org/licenses/by-nc-sa/4.0/": "Q42553662",
        "CC_BY_NC_4_0": "Q24082749", "http://creativecommons.org/licenses/by-nc/4.0/": "Q24082749",
        "CC_BY_NC_3_0": "Q18810331", "http://creativecommons.org/licenses/by-nc/3.0/": "Q18810331",
        "CC_BY_ND_2_5": "Q18810338", "http://creativecommons.org/licenses/by-nd/2.5/": "Q18810338",
        "CC_BY_ND_3_0": "Q18810160", "http://creativecommons.org/licenses/by-nd/3.0/": "Q18810160",
        "CC_BY_ND_4_0": "Q36795408", "http://creativecommons.org/licenses/by-nd/4.0/": "Q36795408",
         "CC_BY_SA_3_0": "Q14946043","http://creativecommons.org/licenses/by-sa/3.0/": "Q14946043",
        "CC_BY_SA_4_0": "Q18199165", "http://creativecommons.org/licenses/by-sa/4.0/": "Q18199165",
        "CC_BY_4_0": "Q18199165", "http://creativecommons.org/licenses/by/4.0/": "Q18199165",
        "CC_BY_3_0": "Q14947546", "http://creativecommons.org/licenses/by/3.0/": "Q14947546",
        "CC0_1_0": "Q18199165", "http://creativecommons.org/publicdomain/zero/1.0/": "Q18199165",
        "https://huh.harvard.edu/access-digital-reproductions-works-public-domain":"Q18199165"
    }
    if not pd.isna(row["license"]) and row["license"] in licenseDict:
        gbifRDF.add((GBIFURI, WDT.P275, URIRef("http://www.wikidata.org/entity/"+licenseDict[row["license"]])))
    else:
        pass
        # print(f"Error for gbifID={row['gbifID']}, license={row['license']}")

    # sex
    sexDict = {
        "FEMALE": "Q43445",
        "MALE": "Q44148"
    }
    if not pd.isna(row["sex"]) and row["sex"] in sexDict:
        gbifRDF.add((GBIFURI, WDT.P21, URIRef("http://www.wikidata.org/entity/"+sexDict[row["sex"]])))
    else:
        pass

    # recordedByID
    if not pd.isna(row["recordedByID"]):
        gbifRDF.add((GBIFURI, WDT.P170, Literal(str(row["recordedByID"]), datatype=XSD.string)))
    else:
        pass

    # associatedSequences
    if not pd.isna(row["associatedSequences"]):
        gbifRDF.add((GBIFURI, WDT.P4333, Literal(str(row["associatedSequences"]), datatype=XSD.string)))
    else:
        pass

    # occurrenceID
    if not pd.isna(row["occurrenceID"]):
        gbifRDF.add((GBIFURI, DC.identifier, Literal(str(row["occurrenceID"]), datatype=XSD.string)))

    # publisher
    if row["rightsHolder"] in instituteQID.keys():
        gbifRDF.add((GBIFURI, WDT.P123, URIRef("http://www.wikidata.org/entity/"+instituteQID[row["rightsHolder"]])))
        gbifRDF.add((URIRef("http://www.wikidata.org/entity/"+instituteQID[row["rightsHolder"]]), RDFS.label, Literal(row["rightsHolder"], lang="en")))

    # kingfom
    if not pd.isna(row["kingdom_y"]):
        gbifRDF.add((GBIFURI, WDT.P910, URIRef(row["kingdom_y"]))) #chapter
        gbifRDF.add((URIRef(row["kingdom_y"]), RDFS.label, Literal(row["kingdomname"], lang="en")))
    else:
        pass

    #Wiki article
    if not pd.isna(row["article"]):
        gbifRDF.add((GBIFURI, DC.description, URIRef(row["article"]))) #chapter
    else:
        pass

87it [00:00, 2816.79it/s]


In [105]:
df_image = pd.read_csv('data/0002020-240626123714530/multimedia.txt', sep='\t', on_bad_lines = 'warn')
for index, row in df_image.iterrows():
    GBIFURI = URIRef("https://gbif.semscape.org/occurrence/"+str(row["gbifID"]))
    mediaURI = BNode()
    if row["type"] == "StillImage":
        gbifRDF.add((mediaURI, WDT.P31, WD.Q17538176))
    elif row["type"] == "Sound":
        gbifRDF.add((mediaURI, WDT.P31, WD.Q11461))
    gbifRDF.add((mediaURI, DCTERMS.format, Literal(row["format"], datatype=XSD.string)))
    if not pd.isna(row["license"]) and row["license"] in licenseDict:
        gbifRDF.add((mediaURI, WDT.P275, URIRef("http://www.wikidata.org/entity/"+licenseDict[row["license"]])))
    else:
        pass
    gbifRDF.add((mediaURI, WDT.P2699, Literal(row["identifier"], datatype=XSD.string)))
    gbifRDF.add((mediaURI, WDT.P361, GBIFURI))

In [106]:
wdtaxqid

{'5289739': 'http://www.wikidata.org/entity/Q4925284',
 '2704745': 'http://www.wikidata.org/entity/Q7579252',
 '2717421': 'http://www.wikidata.org/entity/Q15584858',
 '2702504': 'http://www.wikidata.org/entity/Q24700355',
 '2705068': 'http://www.wikidata.org/entity/Q10912779',
 '2708680': 'http://www.wikidata.org/entity/Q11075670',
 '2702854': 'http://www.wikidata.org/entity/Q15508551',
 '2702852': 'http://www.wikidata.org/entity/Q15508581',
 '2702522': 'http://www.wikidata.org/entity/Q15516186',
 '5291147': 'http://www.wikidata.org/entity/Q15550242',
 '2705275': 'http://www.wikidata.org/entity/Q33913',
 '2705758': 'http://www.wikidata.org/entity/Q3219428',
 '2705656': 'http://www.wikidata.org/entity/Q3367780',
 '2706277': 'http://www.wikidata.org/entity/Q4790962',
 '2706079': 'http://www.wikidata.org/entity/Q15504478',
 '2720622': 'http://www.wikidata.org/entity/Q15557622',
 '2712691': 'http://www.wikidata.org/entity/Q15564658',
 '2718767': 'http://www.wikidata.org/entity/Q21265100',


In [107]:
# Define the URI prefix you are interested in
prefix = "http://www.wikidata.org/entity/"

# Initialize a set to hold all matching URIs
wikidata_uris = set()

# Iterate through all nodes in the graph, both subjects and objects
for node in gbifRDF.all_nodes():
    if isinstance(node, URIRef) and str(node).startswith(prefix):
        wikidata_uris.add(str(node))


In [108]:
gbifRDF.serialize(destination='gbifMontserrat.ttl', format='turtle', encoding="utf-8")

<Graph identifier=N8d51f83dcba44792aa44d81c4e121930 (<class 'rdflib.graph.Graph'>)>

In [109]:
wdGraph = Graph()

# Wrapping wikidata_uris with tqdm to create a progress bar
for wduri in tqdm.tqdm(wikidata_uris, desc="Processing URIs"):
    try:
        print(str(wduri))
        wdGraph.parse(str(wduri), format='turtle')
    except Exception as e:  # It's a good practice to log the exception
        print(f"An error occurred while parsing {wduri}: {e}")
        continue

Processing URIs:   0%|          | 0/41 [00:00<?, ?it/s]

http://www.wikidata.org/entity/Q24700355


Processing URIs:   2%|▏         | 1/41 [00:02<01:55,  2.89s/it]

http://www.wikidata.org/entity/Q15584858


Processing URIs:   5%|▍         | 2/41 [00:04<01:25,  2.19s/it]

...

Processing URIs:  95%|█████████▌| 39/41 [01:40<00:05,  2.60s/it]

http://www.wikidata.org/entity/Q15516186


Processing URIs:  98%|█████████▊| 40/41 [01:41<00:02,  2.36s/it]

http://www.wikidata.org/entity/Q15504478


Processing URIs: 100%|██████████| 41/41 [01:43<00:00,  2.52s/it]


In [110]:
wdGraph.serialize(destination='wikidataMontserrat.ttl', format='turtle', encoding="utf-8")

<Graph identifier=N3cc229e57417463bb9c56ae2f21f9143 (<class 'rdflib.graph.Graph'>)>

In [111]:
query = """
PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    SELECT DISTINCT  ?taxon ?taxonLabel ?publisher ?publisherLabel ?observation ?gbifObservation ?media ?media_url ?license WHERE {
           ?media wdt:P2699 ?media_url ;
                  wdt:P275 ?license ;
                  wdt:P361 ?observation .
          ?observation wdt:P225 ?taxon ;
                       wdt:P854 ?gbifObservation ;
                       wdt:P123 ?publisher .
          ?taxon rdfs:label ?taxonLabel .
          OPTIONAL {?taxon dc:description ?article .}
          ?publisher rdfs:label ?publisherLabel .
            }"""
# Assuming gbifRDF.query(query) has been defined and executed
results = gbifRDF.query(query)
print(results)
taxonpages = {}  # Ensure that taxonpages is defined before the loop

for row in results:
    # Cache commonly used row items to avoid repetitive dictionary lookups and conversions
    taxon_label = str(row["taxonLabel"])
    print(taxon_label)
    publisher_label = str(row["publisherLabel"])

    # If the taxon label is not in taxonpages, initialize it with necessary keys
    if taxon_label not in taxonpages:
        taxonpages[taxon_label] = {
            "wd_uri": str(row["taxon"]),
            "publisher": {},  # Initialize the publisher dictionary here
        }
        # Add the article if it exists in the row
        if "article" in row:
            taxonpages[taxon_label]["article"] = str(row["article"])

    # Ensure the publisher label is initialized in the taxon's publisher dictionary
    if publisher_label not in taxonpages[taxon_label]["publisher"]:
        taxonpages[taxon_label]["publisher"][publisher_label] = {
            "observations": []  # Initialize the observations list here
        }

    # Construct the observation dictionary
    observation = {
        "observation_id": str(row["gbifObservation"]),
        "media": [str(row["media_url"])],  # Initialize media as a list with the media_url
        "license": str(row["license"])
    }
    # Append the observation dictionary to the observations list
    taxonpages[taxon_label]["publisher"][publisher_label]["observations"].append(observation)

# The rest of your processing can continue after this




Cyperus brevifolius
Cyperus brevifolius
Rhynchospora nervosa
Rhynchospora nervosa
Rhynchospora nervosa
Rhynchospora nervosa
Lasiacis divaricata
Axonopus compressus
Rhynchospora nervosa
Rhynchospora nervosa
Eleocharis flavescens
Eriochloa punctata
Cymbopogon citratus
Cyperus sphacelatus
Anthephora hermaphrodita
Lasiacis divaricata
Andropogon bicornis
Cenchrus echinatus
Scleria secans
Cyperus alopecuroides
Cyperus sphacelatus
Coix lacryma-jobi
Aristida adscensionis
Panicum trichoides
Paspalum conjugatum
Panicum trichoides
Aristida adscensionis
Digitaria insularis
Eriochloa punctata
Digitaria insularis
Rhynchospora tenerrima
Fimbristylis complanata
Anthephora hermaphrodita
Anthephora hermaphrodita
Anthephora hermaphrodita
Rhynchospora tenerrima
Cyperus alopecuroides
Cenchrus echinatus
Paspalum fimbriatum
Panicum trichoides
Panicum trichoides
Lasiacis divaricata
Lasiacis sorghoidea
Digitaria insularis
Cyperus sphacelatus
Fimbristylis complanata
Aristida adscensionis
Anthephora hermaphrodit

In [112]:
taxonpages

{'Cyperus brevifolius': {'wd_uri': 'http://www.wikidata.org/entity/Q21265100',
  'publisher': {'Meise Botanic Garden': {'observations': [{'observation_id': 'https://www.gbif.org/occurrence/4876300936',
      'media': ['https://iiif-manifest.oxalis.br.fgov.be/specimen/BR0000027525951V/manifest'],
      'license': 'http://www.wikidata.org/entity/Q18199165'},
     {'observation_id': 'https://www.gbif.org/occurrence/4876300936',
      'media': ['https://oxalis.br.fgov.be/images/V/BR0/000/027/525/951/BR0000027525951V.jpg'],
      'license': 'http://www.wikidata.org/entity/Q18199165'}]}}},
 'Rhynchospora nervosa': {'wd_uri': 'http://www.wikidata.org/entity/Q15555289',
  'publisher': {'Meise Botanic Garden': {'observations': [{'observation_id': 'https://www.gbif.org/occurrence/4876284841',
      'media': ['https://iiif-manifest.oxalis.br.fgov.be/specimen/BR0000026554150V/manifest'],
      'license': 'http://www.wikidata.org/entity/Q18199165'},
     {'observation_id': 'https://www.gbif.org/occ

In [113]:
import shutil
import os
shutil.rmtree('./taxa/')
os.makedirs(os.path.dirname('./taxa/'), exist_ok=True)


In [ ]:
def resolve_license_label(license_uri):
    """Convert a Wikidata license URI to a human-readable label."""
    license_map = {
        'Q18199165': ('CC BY 4.0',       'https://creativecommons.org/licenses/by/4.0/'),
        'Q20007257': ('CC BY-SA 4.0',    'https://creativecommons.org/licenses/by-sa/4.0/'),
        'Q6938433':  ('CC0 1.0',         'https://creativecommons.org/publicdomain/zero/1.0/'),
        'Q19068220': ('CC BY-NC 4.0',    'https://creativecommons.org/licenses/by-nc/4.0/'),
        'Q26952697': ('CC BY-NC-SA 4.0', 'https://creativecommons.org/licenses/by-nc-sa/4.0/'),
        'Q35254':    ('CC BY-SA 3.0',    'https://creativecommons.org/licenses/by-sa/3.0/'),
        'Q24082749': ('CC BY-SA 4.0',    'https://creativecommons.org/licenses/by-sa/4.0/'),
        'Q62322Plain': ('CC BY 4.0',     'https://creativecommons.org/licenses/by/4.0/'),
    }
    s = str(license_uri)
    if 'wikidata.org/entity/Q' in s:
        qid = s.split('/')[-1]
        if qid in license_map:
            label, url = license_map[qid]
            return label, url
        return qid, s
    # Already a CC URL
    s_lower = s.lower()
    if 'publicdomain/zero' in s_lower:   return 'CC0 1.0', s
    if 'by-nc-sa' in s_lower:           return 'CC BY-NC-SA 4.0', s
    if 'by-nc' in s_lower:              return 'CC BY-NC 4.0', s
    if 'by-sa' in s_lower:              return 'CC BY-SA 4.0', s
    if 'by' in s_lower:                 return 'CC BY 4.0', s
    return s, s

print('resolve_license_label() defined.')


In [114]:
for taxon in taxonpages.keys():
    mdFile = MdUtils(file_name='taxa/'+ taxon.replace(' ', '_'), title=taxon+' ('+taxonpages[taxon]['wd_uri'].replace('http://www.wikidata.org/entity/', '')+')')

    if URIRef(row['taxon']) in taxoninformation.keys():
        if 'wikiintroduction' in taxoninformation[URIRef(row['taxon'])].keys():
            mdFile.new_line(taxoninformation[URIRef(row['taxon'])]['wikiintroduction'])
            mdFile.new_line(f"Read more on [English Wikipedia](https://en.wikipedia.org/page/{row['taxonLabel'].replace('', '_')})")
    mdFile.new_line(f"Visualize Wikidata on [Scholia](https://scholia.toolforge.org/taxon/{taxonpages[taxon]['wd_uri'].replace('http://www.wikidata.org/entity/', '')})")
    for organisation in taxonpages[taxon]['publisher'].keys():
        mdFile.new_header(level=1, title=organisation)
        for observation in taxonpages[taxon]['publisher'][organisation]['observations']:
            mdFile.new_line(f"Observation: [{observation['observation_id']}]({observation['observation_id']})")
            license_label, license_url = resolve_license_label(observation['license'])
            mdFile.new_line(f"License: [{license_label}]({license_url})")
            for media in observation['media']:
                if media != 'nan':
                    mdFile.new_line(f"![{observation['observation_id']}]({media.replace('square', 'medium')})")
    mdFile.create_md_file()


In [115]:
organization_to_taxa = {}

for taxon, data in taxonpages.items():
    for organisation, org_data in data['publisher'].items():
        if organisation not in organization_to_taxa:
            organization_to_taxa[organisation] = []
        organization_to_taxa[organisation].append({
            'taxon': taxon,
            'taxon_uri': data['wd_uri'],
            'observations': org_data['observations']
        })

for organisation, taxa in organization_to_taxa.items():
    mdFile = MdUtils(file_name='organisation/' + organisation.replace(' ', '_'), title=organisation)

    for taxon_data in taxa:
        taxon = taxon_data['taxon']
        taxon_uri = taxon_data['taxon_uri']
        observations = taxon_data['observations']

        mdFile.new_header(level=1, title=taxon + ' (' + taxon_uri.replace('http://www.wikidata.org/entity/', '') + ')')

        if URIRef(taxon_uri) in taxoninformation:
            taxon_info = taxoninformation[URIRef(taxon_uri)]
            if 'wikiintroduction' in taxon_info:
                mdFile.new_line(taxon_info['wikiintroduction'])
                mdFile.new_line(f"Read more on [English Wikipedia](https://en.wikipedia.org/page/{taxon.replace(' ', '_')})")

        mdFile.new_line(f"Visualize Wikidata on [Scholia](https://scholia.toolforge.org/taxon/{taxon_uri.replace('http://www.wikidata.org/entity/', '')})")

        for observation in observations:
            mdFile.new_line(f"Observation: [{observation['observation_id']}]({observation['observation_id']})")
            license_label, license_url = resolve_license_label(observation['license'])
            mdFile.new_line(f"License: [{license_label}]({license_url})")
            for media in observation['media']:
                if media != 'nan':
                    mdFile.new_line(f"![{observation['observation_id']}]({media.replace('square', 'medium')})")

    mdFile.create_md_file()


In [89]:
import yaml

with open(r'_toc.yml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    species = yaml.load(file, Loader=yaml.FullLoader)

species["parts"][0]["chapters"] = []
for file in os.listdir('./organisation/'):
    species["parts"][0]["chapters"].append({'file': 'organisation/'+file})

species["parts"][1]["chapters"] = []
for file in os.listdir('./taxa/'):
    species["parts"][1]["chapters"].append({'file': 'taxa/'+file})
with open(r'_toc.yml', 'w+') as file:
    documents = yaml.dump(species, file)

## Below is for reference only

In [72]:
# Assuming `notFound` is your dictionary
data = []  # List to hold data for DataFrame

for institute, details in notFound.items():
    try:
        # Access the first result for the current institute
        if len(details["results"]) == 0:
            # If no results are found, append a dictionary with the institute and an error message
            print(f"No results found for {institute}")
            data.append({
                "Institute": "FOUT" + institute,
                "Error": "No results found"
            })
            continue
        first_result = details["results"][0]

        # Append a dictionary with the required information to the data list
        data.append({
            "Institute": institute,
            "Key": first_result.get("key", None),
            "Title": first_result.get("title", None),
            "Homepage": first_result.get("homepage", None),
            "City": first_result.get("city", None),
            "Country": first_result.get("country", None),
            "PostalCode": first_result.get("postalCode", None),
            "Address": first_result.get("address", None),
            "Phone": first_result.get("phone", None),
            "Email": first_result.get("email", None),
            "Endpoints": first_result.get("endpoints", None)
        })
    except KeyError as e:
        # If some data is not found, append a dictionary with the institute and an error message
        print(f"Error retrieving data for {institute}: {e}")
        data.append({
            "Institute": "FOUT" + institute,
            "Error": details["results"]
        })

# Create DataFrame from the data list
dfinstitutestoBuild = pd.DataFrame(data)
dfinstitutestoBuild


No results found for Delmaude C Ryan
No results found for slauren
No results found for gorillarebecca
No results found for monkeyjodey


,Institute,Error
0,FOUTDelmaude C Ryan,No results found
1,FOUTslauren,No results found
2,FOUTgorillarebecca,No results found
3,FOUTmonkeyjodey,No results found


In [73]:
df.groupby('license').count()

,gbifID,accessRights,bibliographicCitation,language,modified,publisher,references,rightsHolder,type,institutionID,...,publishedByGbifRegion,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name,iucnRedListCategory
license,,,,,,,,,,,,,,,,,,,,,
CC0_1_0,74,0,0,19,53,0,47,46,19,45,...,74,27,27,27,27,0,0,0,0,73
CC_BY_4_0,4,0,0,0,3,0,0,2,2,2,...,4,2,2,2,2,0,0,0,0,4
CC_BY_NC_4_0,9,0,0,0,8,0,8,8,1,2,...,9,7,7,7,7,0,0,0,0,9


In [74]:
df.groupby('iucnRedListCategory').count()

,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,...,gbifRegion,publishedByGbifRegion,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name
iucnRedListCategory,,,,,,,,,,,,,,,,,,,,,
LC,41,0,0,11,41,28,0,24,25,13,...,41,41,14,14,14,14,0,0,0,0
NE,45,0,0,8,45,35,0,30,30,9,...,45,45,21,21,21,21,0,0,0,0


In [75]:
df.groupby('higherClassification').count()

,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,...,publishedByGbifRegion,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name,iucnRedListCategory
higherClassification,,,,,,,,,,,,,,,,,,,,,
Cyperaceae,1,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
POACEAE,1,0,0,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
"Plantae, Monocotyledonae, Poales, Cyperaceae, Cyperoideae",7,0,0,0,7,7,0,0,0,0,...,7,0,0,0,0,0,0,0,0,7
"Plantae, Monocotyledonae, Poales, Poaceae, Aristidoideae",1,0,0,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
"Plantae, Monocotyledonae, Poales, Poaceae, Panicoideae",18,0,0,0,18,18,0,0,0,0,...,18,1,1,1,1,0,0,0,0,18
Plantae|Spermatophyta|Tracheophyta|Magnoliophyta|Monocots|Commelinids|Poales|Cyperaceae|Cyperus,2,0,0,0,2,2,0,2,2,0,...,2,0,0,0,0,0,0,0,0,2
Plantae|Spermatophyta|Tracheophyta|Magnoliophyta|Monocots|Commelinids|Poales|Cyperaceae|Fimbristylis,1,0,0,0,1,1,0,1,1,0,...,1,0,0,0,0,0,0,0,0,1
Plantae|Spermatophyta|Tracheophyta|Magnoliophyta|Monocots|Commelinids|Poales|Cyperaceae|Scleria,1,0,0,0,1,1,0,1,1,0,...,1,0,0,0,0,0,0,0,0,1
Plantae|Spermatophyta|Tracheophyta|Magnoliophyta|Monocots|Commelinids|Poales|Poaceae|Anthephora,1,0,0,0,1,1,0,1,1,0,...,1,0,0,0,0,0,0,0,0,1


In [76]:
df.groupby('species').count()[["higherClassification"]]

,higherClassification
species,
Andropogon bicornis,1
Anthephora hermaphrodita,4
Aristida adscensionis,1
Axonopus compressus,0
Bambusa vulgaris,0
Cenchrus echinatus,1
Coix lacryma-jobi,1
Cymbopogon citratus,1
Cyperus alopecuroides,2


In [77]:
df.groupby('species').count()

,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,...,publishedByGbifRegion,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name,iucnRedListCategory
species,,,,,,,,,,,,,,,,,,,,,
Andropogon bicornis,2,0,0,1,2,1,0,1,1,1,...,2,0,0,0,0,0,0,0,0,2
Anthephora hermaphrodita,5,0,0,1,5,4,0,2,2,1,...,5,1,1,1,1,0,0,0,0,5
Aristida adscensionis,3,0,0,2,3,1,0,2,2,2,...,3,2,2,2,2,0,0,0,0,3
Axonopus compressus,1,0,0,0,1,1,0,1,1,0,...,1,1,1,1,1,0,0,0,0,1
Bambusa vulgaris,1,0,0,0,1,1,0,1,1,0,...,1,1,1,1,1,0,0,0,0,1
Cenchrus echinatus,4,0,0,1,4,3,0,3,3,1,...,4,2,2,2,2,0,0,0,0,4
Coix lacryma-jobi,2,0,0,1,2,1,0,1,1,1,...,2,0,0,0,0,0,0,0,0,2
Cymbopogon citratus,2,0,0,0,2,2,0,1,1,0,...,2,1,1,1,1,0,0,0,0,2
Cyperus alopecuroides,3,0,0,1,3,2,0,1,1,1,...,3,0,0,0,0,0,0,0,0,3


In [78]:
df.groupby('sex').count()

,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,...,publishedByGbifRegion,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name,iucnRedListCategory
sex,,,,,,,,,,,,,,,,,,,,,


In [79]:
df.groupby('taxonRank').count()

,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,...,publishedByGbifRegion,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name,iucnRedListCategory
taxonRank,,,,,,,,,,,,,,,,,,,,,
SPECIES,86,0,0,19,86,63,0,54,55,22,...,86,35,35,35,35,0,0,0,0,86
VARIETY,1,0,0,0,1,1,0,1,1,0,...,1,1,1,1,1,0,0,0,0,0


In [80]:
df.groupby('publishingCountry').count()

,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,...,publishedByGbifRegion,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name,iucnRedListCategory
publishingCountry,,,,,,,,,,,,,,,,,,,,,
BE,2,0,0,0,2,2,0,0,2,2,...,2,2,2,2,2,0,0,0,0,2
DE,1,0,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1
GB,14,0,0,0,14,12,0,11,11,0,...,14,10,10,10,10,0,0,0,0,14
US,70,0,0,19,70,50,0,43,43,20,...,70,24,24,24,24,0,0,0,0,69


In [81]:
result_df.groupby("kingdomname").count()

,taxonId,taxonname,taxon,kingdom,article,genericName,specificEpithet
kingdomname,,,,,,,
plant,28,28,28,28,28,28,28


In [82]:
merged_df[merged_df['taxonId']==""][['taxonKey', 'taxonId', 'taxonname', 'taxon', 'genericName', 'specificEpithet']].drop_duplicates()

,taxonKey,taxonId,taxonname,taxon,genericName,specificEpithet


In [83]:
    wdtaxqid

{'5289739': 'http://www.wikidata.org/entity/Q4925284',
 '2704745': 'http://www.wikidata.org/entity/Q7579252',
 '2717421': 'http://www.wikidata.org/entity/Q15584858',
 '2702504': 'http://www.wikidata.org/entity/Q24700355',
 '2705068': 'http://www.wikidata.org/entity/Q10912779',
 '2708680': 'http://www.wikidata.org/entity/Q11075670',
 '2702854': 'http://www.wikidata.org/entity/Q15508551',
 '2702852': 'http://www.wikidata.org/entity/Q15508581',
 '2702522': 'http://www.wikidata.org/entity/Q15516186',
 '5291147': 'http://www.wikidata.org/entity/Q15550242',
 '2705275': 'http://www.wikidata.org/entity/Q33913',
 '2705758': 'http://www.wikidata.org/entity/Q3219428',
 '2705656': 'http://www.wikidata.org/entity/Q3367780',
 '2706277': 'http://www.wikidata.org/entity/Q4790962',
 '2702936': 'http://www.wikidata.org/entity/Q15504902',
 '2715182': 'http://www.wikidata.org/entity/Q15535602',
 '2706079': 'http://www.wikidata.org/entity/Q15504478',
 '2720622': 'http://www.wikidata.org/entity/Q15557622',


In [84]:
df_image.groupby("license").count()

,gbifID,type,format,identifier,references,title,description,source,audience,created,creator,contributor,publisher,rightsHolder
license,,,,,,,,,,,,,,
"Partial images provided by this server are released under the Creative Commons cc-by-sa 3.0 (generic) licence [https://creativecommons.org/licenses/by-sa/3.0/de/]. Please credit images to BGBM following our citation guidelines [https://ww2.bgbm.org/Herbarium/cite.cfm]. If you would like to use images in a format or resolution which is not provided here, please contact us (d.roepert[at]bgbm.org).",2,1,2,2,2,0,0,0,0,0,0,0,0,0
http://creativecommons.org/licenses/by-nc/4.0/,8,8,8,8,7,1,1,0,0,7,8,0,7,8
http://creativecommons.org/licenses/by-sa/4.0/,4,4,4,4,0,4,4,0,0,0,4,0,0,4
http://creativecommons.org/licenses/by/4.0/,21,21,20,21,1,1,0,0,0,0,19,0,0,21
http://creativecommons.org/publicdomain/zero/1.0/,55,55,55,55,28,27,27,27,0,28,55,0,55,28


In [85]:
df_image.groupby("type").count()

,gbifID,format,identifier,references,title,description,source,audience,created,creator,contributor,publisher,license,rightsHolder
type,,,,,,,,,,,,,,
InteractiveResource,2,2,2,0,2,2,0,0,0,2,0,0,2,2
StillImage,87,86,87,37,31,30,27,0,35,84,0,62,87,59


In [86]:
df_image[df_image["type"]=="Sound"]

,gbifID,type,format,identifier,references,title,description,source,audience,created,creator,contributor,publisher,license,rightsHolder


In [87]:
df_image.groupby('identifier').count()

,gbifID,type,format,references,title,description,source,audience,created,creator,contributor,publisher,license,rightsHolder
identifier,,,,,,,,,,,,,,
http://n2t.net/ark:/65665/m30612c7f4-df88-4fbb-b508-cb574264bef3,1,1,1,0,1,1,1,0,0,1,0,1,1,0
http://n2t.net/ark:/65665/m3140f66a1-e14d-48e1-b859-a9cd019ffd21,1,1,1,0,1,1,1,0,0,1,0,1,1,0
http://n2t.net/ark:/65665/m31c9184db-f3b6-498c-b1fd-6e7356c686fb,1,1,1,0,1,1,1,0,0,1,0,1,1,0
http://n2t.net/ark:/65665/m31d2ae640-4b6b-4399-a7d7-50e02ec7a5ae,1,1,1,0,1,1,1,0,0,1,0,1,1,0
http://n2t.net/ark:/65665/m31fc6a10b-0312-48a6-b36a-e61a21a54136,1,1,1,0,1,1,1,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://inaturalist-open-data.s3.amazonaws.com/photos/355235906/original.jpg,1,1,1,1,0,0,0,0,1,1,0,1,1,1
https://inaturalist-open-data.s3.amazonaws.com/photos/362637316/original.jpg,1,1,1,1,0,0,0,0,1,1,0,1,1,1
https://inaturalist-open-data.s3.amazonaws.com/photos/63505592/original.jpeg,1,1,1,1,0,0,0,0,1,1,0,1,1,1


In [71]:
len(gbifRDF.all_nodes())

0

In [ ]:
len (gbifRDF.all_nodes())

<!-- Legacy scratch cell — disabled. Superseded by cells 30/31 above. -->
<!--
```python
query = """
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    SELECT DISTINCT ?taxon ?taxonLabel WHERE {
           ?observation wdt:P225 ?taxon .
           ?taxon ?p ?taxonLabel .
            }"""
results = gbifRDF.query(query)
for row in results:
    print(row["taxon"], row["taxonLabel"])
    mdFile = mdutils.MdUtils(file_name='taxa/'+ row["taxonLabel"].replace(" ", "_"), title=row["taxonLabel"]+" ("+row["taxon"]+")")
    if URIRef(row["taxon"]) in taxoninformation.keys():
        if "wikiintroduction" in taxoninformation[URIRef(row["taxon"])].keys():
            mdFile.new_line(taxoninformation[URIRef(row["taxon"])]["wikiintroduction"])
            mdFile.new_line(f"Read more on [English Wikipedia](https://en.wikipedia.org/page/{row['taxonLabel'].replace('' , '_')})")
        mdFile.new_line(f"Visualize Wikidata on [Scholia](https://scholia.toolforge.org/taxon/{row['taxon'].replace('http://www.wikidata.org/entity/', '')})")```
-->


<!-- Legacy scratch cell — disabled. References undefined variables `table` and `verified`. -->
<!--
```python
import mdutils
for taxon in table.keys():
    if table[taxon]["taxon_name"] in verified:
        mdFile = mdutils.MdUtils(file_name='new_articles/'+table[taxon]["taxon_name"].replace(" ", "_"),title=table[taxon]["taxon_name"])
        mdFile.new_line(f"[iNaturalist taxon id: {taxon}](https://www.inaturalist.org/taxa/{taxon})")
        for image in table[taxon]["photos"]:
            mdFile.new_line(f"![{table[taxon]['taxon_name']}]({image.replace('square', 'medium')})")
        mdFile.create_md_file()```
-->


<!-- empty cell -->